## Bayer Case Study - Snaffleflax
#### Sarah Beth Sanchini  

###### **Goal**: Combine JSON and CSV data into a single file

**Specifications**: Include columns for: 

> 1. Market Share
2. Lagged X-Month Average of Market Share
3. Lagged X-Month Sum of Weighted Events

###### Start with importing all libraries 

In [47]:
import pandas as pd
import json
import csv
import numpy as np
import os
import glob
import datetime
pd.options.mode.chained_assignment = None

###### Portion of code for user to set variables

In [48]:
#directory filepaths
in_filepath = "/Users/sarahbeth/Desktop/Bayer/data"
out_filepath = "/Users/sarahbeth/Desktop/Bayer/"

#column specifications: Lagged X-Month Average of Market Share 
### How many months would you like to lag? ###  
x1 = 3 

#column specifications: Lagged X-Month Sum of Weighted Events
### How many months would you like to lag? ###  
x2 = 1

###### Read in multiple JSON files by parsing the directory
> Each JSON files is opened and saved to a list, which is then combined into one dataframe object

In [49]:
#create list of all files ending with .json
file_list = glob.glob(in_filepath + "/*.json")
#read in CRM data
df_crm = pd.read_csv(in_filepath +'/crm_data.csv')

In [50]:
#open JSON files
dfs = [] # an empty list to store the data frames
for file in file_list:
    json_file1 = open(file)
    json_data1 = json.load(json_file1)
    df1 = pd.read_json(json_data1[0]) # json was in a -1 list (double-encoded) so specify index
    dfs.append(df1) # append the data frame to the list

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list
df.head()

,acct_id,product_name,date,unit_sales,created_at
0,f51ae438-3ada-405b-ad1e-f1d9bee81c62,Globbrin,2019-01-01,2,2019-01-26 15:00:00
1,1e87bd4f-067d-4110-bae7-33a45be6e073,Snaffleflax,2019-01-01,4,2019-01-23 13:00:00
2,14a34247-8fc4-4654-897b-d850c719c17e,Snaffleflax,2019-01-01,1,2019-01-15 05:00:00
3,deaa8812-4099-4163-8023-4b14d04dc1f6,Globberin,2019-01-01,2,2019-01-17 17:00:00
4,7f9ebdd6-f7a4-4fe1-9f6a-d6ad0e77308f,Globbrin,2019-01-01,1,2019-01-08 07:00:00


###### Data Cleaning
> Product Name column should only have 4 unique names

In [51]:
#data exploration - product name
df.product_name.unique()

array(['Globbrin', 'Snaffleflax', ' Globberin', 'Vorbulon', 'vorbulon.',
       'Beebliz%C3%B6x', 'Globberin', 'Snafulopromazide-b (Snaffleflax)',
       'Beeblizox'], dtype=object)

In [52]:
#clean product_name
df = df.replace({"Globbrin":"Globberin", " Globberin":"Globberin","vorbulon.":"Vorbulon","Beebliz%C3%B6x":"Beeblizox","Snafulopromazide-b (Snaffleflax)":"Snaffleflax"})

###### Create column for Market Share 
> 1. Calculate total units sold over 18 month period
2. Calculate each row entry's market share (row's units sold / total units sold)

In [53]:
#calculate each row's market share
total_MS = df['unit_sales'].sum()
df['marketshare'] = df['unit_sales']/total_MS
df.head()

,acct_id,product_name,date,unit_sales,created_at,marketshare
0,f51ae438-3ada-405b-ad1e-f1d9bee81c62,Globberin,2019-01-01,2,2019-01-26 15:00:00,0.000417
1,1e87bd4f-067d-4110-bae7-33a45be6e073,Snaffleflax,2019-01-01,4,2019-01-23 13:00:00,0.000835
2,14a34247-8fc4-4654-897b-d850c719c17e,Snaffleflax,2019-01-01,1,2019-01-15 05:00:00,0.000209
3,deaa8812-4099-4163-8023-4b14d04dc1f6,Globberin,2019-01-01,2,2019-01-17 17:00:00,0.000417
4,7f9ebdd6-f7a4-4fe1-9f6a-d6ad0e77308f,Globberin,2019-01-01,1,2019-01-08 07:00:00,0.000209


###### Create column for Lagged X-Month Average of Market Share
> 1. Calculate the number of days for the lagged period as Python does not support rolling month calculations
2. For each row, calculate the rolling average of the specified time frame

In [54]:
#sort data
df = df.sort_values(by="created_at")
df.head()

,acct_id,product_name,date,unit_sales,created_at,marketshare
16,6dbe112e-d8e1-4bc5-82c1-b18fad3da004,Globberin,2019-01-01,5,2019-01-01 19:00:00,0.001044
6,836fcb85-7ef5-44b6-b00a-082185971863,Snaffleflax,2019-01-01,5,2019-01-03 08:00:00,0.001044
66,aac0916b-9a0f-45f3-9110-66dfbeb54530,Globberin,2019-01-01,5,2019-01-04 02:00:00,0.001044
78,51b0fd52-5e62-454b-9c46-e4e6fb7a5c1b,Globberin,2019-01-01,4,2019-01-04 05:00:00,0.000835
61,b028373e-1a33-42ef-8e88-822c1269a0b5,Vorbulon,2019-01-01,2,2019-01-04 11:00:00,0.000417


In [55]:
## need to convert months to days
in_x1 = str(x1*30)
in_x1 = in_x1 + "D"

#calculate rolling sum and count of occurences in the month
df2 = df.set_index('created_at').rolling(in_x1)['marketshare'].agg({'sum','count'})
#calculation
df2['lagged x-month avg marketshare'] = df2['sum']/df2['count']
df2.head()

,sum,count,lagged x-month avg marketshare
created_at,,,
2019-01-01 19:00:00,0.001044,1.0,0.001044
2019-01-03 08:00:00,0.002087,2.0,0.001044
2019-01-04 02:00:00,0.003131,3.0,0.001044
2019-01-04 05:00:00,0.003966,4.0,0.000991
2019-01-04 11:00:00,0.004383,5.0,0.000877


In [56]:
#merge dataframes together
df_merged = pd.merge(df,df2, on=['created_at'])
df_merged.head()

,acct_id,product_name,date,unit_sales,created_at,marketshare,sum,count,lagged x-month avg marketshare
0,6dbe112e-d8e1-4bc5-82c1-b18fad3da004,Globberin,2019-01-01,5,2019-01-01 19:00:00,0.001044,0.001044,1.0,0.001044
1,836fcb85-7ef5-44b6-b00a-082185971863,Snaffleflax,2019-01-01,5,2019-01-03 08:00:00,0.001044,0.002087,2.0,0.001044
2,aac0916b-9a0f-45f3-9110-66dfbeb54530,Globberin,2019-01-01,5,2019-01-04 02:00:00,0.001044,0.003131,3.0,0.001044
3,51b0fd52-5e62-454b-9c46-e4e6fb7a5c1b,Globberin,2019-01-01,4,2019-01-04 05:00:00,0.000835,0.003966,4.0,0.000991
4,b028373e-1a33-42ef-8e88-822c1269a0b5,Vorbulon,2019-01-01,2,2019-01-04 11:00:00,0.000417,0.004383,5.0,0.000877


### CRM Data Pre-Processing

###### Create column for Lagged X-Month Weighted Sum of Events
> **Specifications**: Column will be a comma separated list of 3 values with the following format:   
(F2F, Group Call, Workplace Events) 

> **Calculation**: 
>1. Calculate the total number of events
2. Calculate the total number of events per event type
3. Calculate weights for each event type (number of particular event type / sum of all events)
4. Calculate weighted value of each type of event *per day* (daily sum of particular event * event weight 

In [57]:
#covert date to datetime format 
df_crm['date']= pd.to_datetime(df_crm['date'])
#sort data
df_crm = df_crm.sort_values(by="date")
df_crm = df_crm.reset_index(drop=True)
df_crm.head()

,acct_id,event_type,date
0,b028373e-1a33-42ef-8e88-822c1269a0b5,f2f,2019-01-01
1,c1ca79b6-c9e2-45cb-ab2e-32b3086a4d9f,workplace event,2019-01-01
2,11bf6770-0394-4c43-ad42-f276e3a5d95c,f2f,2019-01-01
3,9864cbfb-6165-4a37-b2d1-9ba53ea15ae6,f2f,2019-01-01
4,aacf794a-4205-472b-b56f-2fad7fe16894,group call,2019-01-01


###### Calculate values for each date
>There are 508 unique dates in the file
1. Aggregate different event types per day
2. Calculate total number of events per day
3. Calculate the weights of each type of event based on given timeframe (lag)

In [58]:
#groupby event type
df_gp = df_crm.groupby(['date'])['event_type'].value_counts().unstack().fillna(0).astype(int)
df_gp.reset_index(inplace = True)

In [59]:
#create total_events column
for i in df_gp.index:
    df_gp['total_events'] = df_gp['f2f'] + df_gp ['group call'] + df_gp['workplace event']
df_gp.head()

event_type,date,f2f,group call,workplace event,total_events
0,2019-01-01,3,1,1,5
1,2019-01-02,1,2,0,3
2,2019-01-03,0,1,1,2
3,2019-01-04,0,2,3,5
4,2019-01-05,1,3,1,5


In [60]:
#set event_date as index to perform rolling operations
df_gp.rename(columns={'date': 'event_date'}, inplace=True)
df_gp = df_gp.set_index('event_date')

In [61]:
#set input lag: need to convert months to days
in_x2 = str((x2+1)*30)
in_x2 = in_x2 + "D"

In [62]:
#calculate weighted sum of each type of event:
#Example: #f2f events * (#f2fevents / #total events)

column_names = ["f2f_weighted", "group_weighted", "workplace_weighted"]
df_gpd = pd.DataFrame(columns = column_names)

df_gpd['f2f_weighted'] = (df_gp['f2f'].rolling(window=in_x2).sum())* ((df_gp['f2f'].rolling(window=in_x2).sum())/(df_gp['total_events'].rolling(window=in_x2).sum())) 
df_gpd['group_weighted'] = (df_gp['group call'].rolling(window=in_x2).sum())* ((df_gp['group call'].rolling(window=in_x2).sum())/(df_gp['total_events'].rolling(window=in_x2).sum())) 
df_gpd['workplace_weighted'] = (df_gp['workplace event'].rolling(window=in_x2).sum())* ((df_gp['workplace event'].rolling(window=in_x2).sum())/(df_gp['total_events'].rolling(window=in_x2).sum()))

#join the 3 weighted values to create comma separated output column
df_gpd['weighted_values (f2f,grp call, wkplace)'] = df_gpd.apply(lambda x: ','.join(x.dropna().astype(str).values), axis=1)

In [63]:
#reset index and make date column
df_gpd = df_gpd.reset_index()
df_gpd.head()

,event_date,f2f_weighted,group_weighted,workplace_weighted,"weighted_values (f2f,grp call, wkplace)"
0,2019-01-01,1.800000,0.200,0.200000,"1.7999999999999998,0.2,0.2"
1,2019-01-02,2.000000,1.125,0.125000,"2.0,1.125,0.125"
2,2019-01-03,1.600000,1.600,0.400000,"1.6,1.6,0.4"
3,2019-01-04,1.066667,2.400,1.666667,"1.0666666666666667,2.4000000000000004,1.666666..."
4,2019-01-05,1.250000,4.050,1.800000,"1.25,4.05,1.7999999999999998"


###### Create output file
> Merge dataframes together and output as CSV file.  
Inner join on date

In [64]:
#set up columns to merge
df_merged.rename(columns={'created_at':'date1'}, inplace=True)
df_gpd.rename(columns={'event_date':'date1'}, inplace=True)
df_output = pd.concat([df_merged,df_gpd])

df_output = df_output.sort_values(by='date1')
df_output.reset_index(inplace=True)

In [68]:
#cleanup columns to export
df_output.rename(columns={'date1':'created_at'}, inplace=True)
df_output.drop(columns=['index', 'count','sum','f2f_weighted','group_weighted','workplace_weighted'], axis = 1,inplace=True)
df_output.head()

,acct_id,product_name,date,unit_sales,created_at,marketshare,lagged x-month avg marketshare,"weighted_values (f2f,grp call, wkplace)"
0,NaN,NaN,NaT,NaN,2019-01-01 00:00:00,NaN,NaN,"1.7999999999999998,0.2,0.2"
1,6dbe112e-d8e1-4bc5-82c1-b18fad3da004,Globberin,2019-01-01,5.0,2019-01-01 19:00:00,0.001044,0.001044,NaN
2,NaN,NaN,NaT,NaN,2019-01-02 00:00:00,NaN,NaN,"2.0,1.125,0.125"
3,NaN,NaN,NaT,NaN,2019-01-03 00:00:00,NaN,NaN,"1.6,1.6,0.4"
4,836fcb85-7ef5-44b6-b00a-082185971863,Snaffleflax,2019-01-01,5.0,2019-01-03 08:00:00,0.001044,0.001044,NaN


In [71]:
#write to csv
df_output.to_csv (out_filepath +'/final_output.csv')